In [1]:
import numpy as np
import tensorflow as tf
import json

np.set_printoptions(suppress=True, precision=4, linewidth=250)
opts = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)
conf = tf.ConfigProto(gpu_options=opts)
tf.enable_eager_execution(config=conf)

In [2]:
from google.protobuf.json_format import MessageToDict, ParseDict

In [6]:
ls link_bot_data/

250_50_collision.npz                 gz_train_1a98b68431_10_1000_balanced/
250_50_random3.npz                   gz_train_20e94e0e54_100_1000/
5000_50_collision.npz                gz_train_231723146d_100_1000/
gz_debug_d0d7994a2e_1_100/           gz_train_2362dceb2a_100_1000/
gz_debug_d0d7994a2e_5_1000/          gz_train_2362dceb2a_100_1000_balanced/
gz_test_231723146d_50_100/           gz_train_c4ac00afdb_10_1000/
gz_test_231723146d_50_100_balanced/  gz_train_c4ac00afdb_10_1000_balanced/
gz_test_23aedc357f_50_100/           gz_train_d0d7994a2e_100_1000/
gz_test_23aedc357f_50_100_balanced/  gz_train_d0d7994a2e_100_1000_balanced/
gz_test_c4ac00afdb_50_100/           moving_block/
gz_test_c4ac00afdb_50_100_balanced/  multi_link_multi_collision_1.npz
gz_test_d0d7994a2e_50_100/           random/
gz_test_d0d7994a2e_50_100_balanced/  sdfs/
gz_train_1a98b68431_10_1000/         unity_cloth/


In [11]:
filename = "./data/bair/test/traj_0_to_255.tfrecords"
example = next(tf.python_io.tf_record_iterator(filename))
dict_message = MessageToDict(tf.train.Example.FromString(example))
print(sorted(dict_message['features']['feature'].keys()))

['0/action', '0/endeffector_pos', '0/image_aux1/encoded', '0/image_main/encoded', '1/action', '1/endeffector_pos', '1/image_aux1/encoded', '1/image_main/encoded', '10/action', '10/endeffector_pos', '10/image_aux1/encoded', '10/image_main/encoded', '11/action', '11/endeffector_pos', '11/image_aux1/encoded', '11/image_main/encoded', '12/action', '12/endeffector_pos', '12/image_aux1/encoded', '12/image_main/encoded', '13/action', '13/endeffector_pos', '13/image_aux1/encoded', '13/image_main/encoded', '14/action', '14/endeffector_pos', '14/image_aux1/encoded', '14/image_main/encoded', '15/action', '15/endeffector_pos', '15/image_aux1/encoded', '15/image_main/encoded', '16/action', '16/endeffector_pos', '16/image_aux1/encoded', '16/image_main/encoded', '17/action', '17/endeffector_pos', '17/image_aux1/encoded', '17/image_main/encoded', '18/action', '18/endeffector_pos', '18/image_aux1/encoded', '18/image_main/encoded', '19/action', '19/endeffector_pos', '19/image_aux1/encoded', '19/image_ma

In [10]:
# filename = "/home/pmitrano/catkin_ws/src/link_bot/link_bot_data/data/moving_block/train/traj_.tfrecords"
filename = "./link_bot_data/unity_cloth/val/traj_256_to_511.tfrecords"
example = next(tf.python_io.tf_record_iterator(filename))
dict_message = MessageToDict(tf.train.Example.FromString(example))
print(sorted(dict_message['features']['feature'].keys()))

['0/action', '0/endeffector_pos', '0/image_aux1/encoded', '1/action', '1/endeffector_pos', '1/image_aux1/encoded', '10/action', '10/endeffector_pos', '10/image_aux1/encoded', '11/action', '11/endeffector_pos', '11/image_aux1/encoded', '12/action', '12/endeffector_pos', '12/image_aux1/encoded', '13/action', '13/endeffector_pos', '13/image_aux1/encoded', '14/action', '14/endeffector_pos', '14/image_aux1/encoded', '15/action', '15/endeffector_pos', '15/image_aux1/encoded', '16/action', '16/endeffector_pos', '16/image_aux1/encoded', '17/action', '17/endeffector_pos', '17/image_aux1/encoded', '18/action', '18/endeffector_pos', '18/image_aux1/encoded', '19/action', '19/endeffector_pos', '19/image_aux1/encoded', '2/action', '2/endeffector_pos', '2/image_aux1/encoded', '20/action', '20/endeffector_pos', '20/image_aux1/encoded', '21/action', '21/endeffector_pos', '21/image_aux1/encoded', '22/action', '22/endeffector_pos', '22/image_aux1/encoded', '23/action', '23/endeffector_pos', '23/image_aux

In [ ]:
ls data